In [3]:
#Questionable quality
#https://www.youtube.com/watch?v=nTt_ajul8NY
#https://towardsdatascience.com/understanding-encoder-decoder-sequence-to-sequence-model-679e04af4346

#https://towardsdatascience.com/language-translation-with-rnns-d84d43b40571
#http://www.wildml.com/2016/01/attention-and-memory-in-deep-learning-and-nlp/

#Great quality
#https://machinelearningmastery.com/lstm-autoencoders/
#https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html
#https://keras.io/examples/lstm_seq2seq/
#https://www.youtube.com/watch?v=QuELiw8tbx8&feature=youtu.be&list=PL3FW7Lu3i5Jsnh1rnUwq_TcylNr7EkRe6&t=1190
#https://stackoverflow.com/questions/50915634/multilayer-seq2seq-model-with-lstm-in-keras

In [4]:
import os
import pretty_midi
import numpy as np
import math
import random
import matplotlib.pyplot as plt

In [5]:
from __future__ import print_function

import keras

from keras.models import Model
from keras.models import load_model

from keras.layers import Input, LSTM, Dense

Using TensorFlow backend.


In [6]:
import pretty_midi
import argparse
import librosa
import os
import sys

In [7]:
workdir = "C:\\Users\\toend\\Documents\\ITU\\Thesis"
pathToSave = os.path.join(workdir, "MIDIdata\\RESULTS")

frequency = 100
START_OF_SEQUENCE_VALUE = 101
END_OF_SEQUENCE_VALUE = 102

In [8]:
sys.path.append(workdir + '\\Code\\Utils')
from Utils import getDataSets
from Utils import reshape
from Utils import piano_roll_to_pretty_midi

In [9]:
X1, X2, X3, X4 = getDataSets(workdir)
X = np.concatenate((X1, X2, X3, X4))

In [10]:
encoderInputRolls = np.array(X1[:150])
decoderInputRolls = np.array(X2[:150])
decoderTargetRolls = np.array(X2[:150])
#print(decoderTargetRolls[0][0])

In [11]:
#SHIFTING TARGET DATA SO THAT IT STAYS ONE TIMESTEP AHEAD

#SHOULD START AND END TOKEN BE DELETED FOR ENCODER? !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

#DELETING END TOKEN FOR DECODER INPUT
#decoderInputRolls = np.delete(decoderInputRolls, 3001 , 1)

#DELETING START TOKEN FOR TARGET
#decoderTargetRolls = np.delete(decoderTargetRolls, 0, 1)


print(encoderInputRolls.shape)
print(decoderInputRolls.shape)
print(decoderTargetRolls.shape)
#print(decoderTargetRolls[0][0])

(150, 3000, 128)
(150, 3000, 128)
(150, 3000, 128)


In [12]:
batch_size = 64  # Batch size for training.
epochs = 150  # Number of epochs to train for.
latent_dim = 100  # Latent dimensionality of the encoding space.
pianoNodes = 128
learning_rate = 0.02  #No clue, default is 0.01
decay = learning_rate/epochs  #https://www.pyimagesearch.com/2019/07/22/keras-learning-rate-schedules-and-decay/

In [13]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, pianoNodes))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# Discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [14]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, pianoNodes))
# Set up our decoder to return full output sequences,
# and to return internal states as well. Don't use the
# return states in the training model, but use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
#CONSIDER ACTIVATION FUNCTION --> SIGMOID MIGHT BE BETTER FOR NOW
decoder_dense = Dense(pianoNodes, activation='sigmoid')
decoder_outputs = decoder_dense(decoder_outputs)

In [15]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

#Define optimizer
optimizer = keras.optimizers.adam(learning_rate=learning_rate)

# Run training
model.compile(optimizer=optimizer, loss='binary_crossentropy',
              metrics=['accuracy'])
model.fit([encoderInputRolls, decoderInputRolls], decoderTargetRolls,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.1)
# Save model
model.save('LSTMs2sb40l03.h5')

Train on 135 samples, validate on 15 samples
Epoch 1/150
135/135 [==============================] - 111s 823ms/step - loss: 0.4096 - accuracy: 0.8328 - val_loss: 0.0605 - val_accuracy: 0.9839
Epoch 2/150
135/135 [==============================] - 112s 831ms/step - loss: 0.0631 - accuracy: 0.9846 - val_loss: 0.0669 - val_accuracy: 0.9839
Epoch 3/150
135/135 [==============================] - 119s 883ms/step - loss: 0.0640 - accuracy: 0.9846 - val_loss: 0.0555 - val_accuracy: 0.9839
Epoch 4/150
135/135 [==============================] - 124s 922ms/step - loss: 0.0577 - accuracy: 0.9846 - val_loss: 0.0526 - val_accuracy: 0.9839
Epoch 5/150
135/135 [==============================] - 126s 936ms/step - loss: 0.0554 - accuracy: 0.9846 - val_loss: 0.0525 - val_accuracy: 0.9839
Epoch 6/150
135/135 [==============================] - 124s 919ms/step - loss: 0.0539 - accuracy: 0.9846 - val_loss: 0.0519 - val_accuracy: 0.9839
Epoch 7/150
135/135 [==============================] - 127s 944ms/step - 

135/135 [==============================] - 145s 1s/step - loss: 0.0410 - accuracy: 0.9846 - val_loss: 0.0411 - val_accuracy: 0.9839
Epoch 113/150
135/135 [==============================] - 146s 1s/step - loss: 0.0409 - accuracy: 0.9846 - val_loss: 0.0411 - val_accuracy: 0.9839
Epoch 114/150
135/135 [==============================] - 146s 1s/step - loss: 0.0408 - accuracy: 0.9846 - val_loss: 0.0411 - val_accuracy: 0.9839
Epoch 115/150
135/135 [==============================] - 146s 1s/step - loss: 0.0408 - accuracy: 0.9846 - val_loss: 0.0413 - val_accuracy: 0.9839
Epoch 116/150
135/135 [==============================] - 146s 1s/step - loss: 0.0408 - accuracy: 0.9846 - val_loss: 0.0410 - val_accuracy: 0.9839
Epoch 117/150
135/135 [==============================] - 146s 1s/step - loss: 0.0408 - accuracy: 0.9846 - val_loss: 0.0411 - val_accuracy: 0.9839
Epoch 118/150
135/135 [==============================] - 147s 1s/step - loss: 0.0407 - accuracy: 0.9846 - val_loss: 0.0411 - val_accuracy:

In [51]:
# load model
model = load_model('LSTMs2s.h5')
# summarize model.
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 128)    0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 128)    0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 100), (None, 91600       input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 100),  91600       input_2[0][0]                    
                                                                 lstm_1[0][1]               

In [21]:
# Next: inference mode (sampling).
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)



In [22]:
# Decode sequences back to something readable.
# Pick only values above 0.50?

In [23]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    
    # Generate empty target sequence of length 1.
    # Populate the first character of target sequence with the start character.
    #start = np.array([START_OF_SEQUENCE_VALUE for i in range(128)])
    
    start = np.zeros(128)
    target_seq = np.reshape(start, (1, 1, 128))
    
    #Normalizing the START_OF_SEQUENCE_VALUE from 101 to 1.01
    #target_seq = target_seq/100

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_pianoRoll = np.empty((1,1,128))
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a single node
        #sampled_node = np.argmax(output_tokens[0, -1, :])
        
        #Add all nodes
        addedPianoRoll = np.reshape(output_tokens[0, -1, :], (1,1,128))
        decoded_pianoRoll = np.append(decoded_pianoRoll, addedPianoRoll , axis=0)

        # Exit condition: either hit max length
        # or find stop character ---> sampled_node == 102
        if (len(decoded_pianoRoll) > 3000):
            stop_condition = True

        # Update the target sequence (of length 1).
        #target_seq = np.zeros((1, 1, 128))
        #target_seq[0, 0, sampled_token_index] = 1.
        
        target_seq = np.reshape(output_tokens[0, -1, :], (1,1,128))
        
        #Readjusting values to avoid vanishing values
        #target_seq = target_seq*2
        target_seq = convertProbabilitiesToActualPianoRoll(target_seq)
        
        # Update states
        states_value = [h, c]

    return decoded_pianoRoll

In [28]:
#Decide cutoff value for node-on/off --- 0.09?
def convertProbabilitiesToActualPianoRoll(decodedPianoRoll):
    #print(decodedPianoRoll[0])
    pianoRoll = np.where(decodedPianoRoll<0.99, decodedPianoRoll, 0.90)
    #Everything above 17%?
    pianoRoll = np.where(decodedPianoRoll>0.10, decodedPianoRoll, 0)
    #print(pianoRoll[0])
    
    #pianoRoll = np.where(pianoRoll==0, pianoRoll, 80)
    #print(pianoRoll[0])
    #print(pianoRoll.shape)
    return pianoRoll

In [29]:
pianoRolls = np.empty((1, 3000, 128))
for seq_index in range(3):
    # Take one sequence (part of the training set) for trying out decoding.
    print('----------------')
    input_seq = encoderInputRolls[seq_index: seq_index + 1]
    print(input_seq.shape) # (1, 3002, 128)
    decoded_pianoRoll = decode_sequence(input_seq)
    print(decoded_pianoRoll.shape) # (3001, 1 , 128) --> can be changed 
    
    #Reshaping to get rid of 3rd dimension
    decoded_pianoRoll = np.reshape(decoded_pianoRoll, (decoded_pianoRoll.shape[0], 128))
    print(decoded_pianoRoll.shape)
    
    #deleting start of sequence
    decoded_pianoRoll = np.delete(decoded_pianoRoll, 0, 0)
    #print(decoded_pianoRoll.shape)
    
    #converting to actual pianoRoll with only chosen node-on/off
    pianoRoll = decoded_pianoRoll * 2
    pianoRoll = convertProbabilitiesToActualPianoRoll(pianoRoll)
    
    #converting probabilities 0-1 to velocity values 0-100
    pianoRoll = pianoRoll*100
    
    #Converting velocity values to values you can actually hear (from 1.5 to something like 60)
    #decoded_pianoRoll = decoded_pianoRoll*60
    
    
    #Inspecting values generated by the model
    #for i in range(200):
    #    print(pianoRoll[i])
    
    #Reshaping to become an actual pianoRoll
    pianoRoll = reshape(pianoRoll)
    print(pianoRoll.shape)
    
    #Flipping nodes around
    #pianoRoll = np.flip(pianoRoll, 0)
    
    #saving to midi
    pm = piano_roll_to_pretty_midi(pianoRoll, frequency, 0)
    pm.write(pathToSave + "\\LSTM" + str(seq_index) + ".mid")
    
    #Adding pianoroll to list of all pianorolls
    #decodedPianoRolls = np.append(decodedPianoRolls, decoded_pianoRoll, axis=0)
    #pianoRolls = np.append(pianoRolls, pianoRoll, axis=0)

----------------
(1, 3000, 128)
(3001, 1, 128)
(3001, 128)
(128, 3000)
----------------
(1, 3000, 128)
(3001, 1, 128)
(3001, 128)
(128, 3000)
----------------
(1, 3000, 128)
(3001, 1, 128)
(3001, 128)
(128, 3000)


In [30]:
for i in range(200):
    print(decoded_pianoRoll[i])

[2.68220901e-07 3.87430191e-07 2.08616257e-07 2.68220901e-07
 3.27825546e-07 1.49011612e-07 1.49011612e-07 2.08616257e-07
 2.08616257e-07 2.98023224e-07 2.68220901e-07 2.08616257e-07
 2.08616257e-07 2.38418579e-07 2.68220901e-07 1.78813934e-07
 2.98023224e-07 5.06639481e-07 1.19209290e-07 2.08616257e-07
 3.57627869e-07 3.57627869e-07 1.15334988e-05 1.63316727e-04
 1.40833855e-03 1.61409378e-04 1.11407042e-03 2.08407640e-04
 2.44519114e-03 1.51443481e-03 1.87107921e-03 8.58813524e-03
 1.16863847e-03 3.41251493e-03 3.55586410e-03 7.16760755e-03
 8.90082121e-03 1.54715776e-03 6.13924861e-03 3.56295705e-03
 2.93543935e-03 3.97175550e-03 4.21449542e-03 6.78312778e-03
 1.10191107e-03 4.26974893e-03 3.09357047e-03 4.49910760e-03
 1.08854771e-02 3.25587392e-03 1.24017596e-02 1.21772587e-02
 3.83329391e-03 3.63537669e-03 5.51989675e-03 2.67060101e-02
 6.79492950e-03 4.25344706e-03 5.90354204e-03 5.73834777e-03
 1.32150948e-02 2.35110521e-03 1.02259815e-02 9.86590981e-03
 2.05284357e-03 3.687322

In [39]:
print(pianoRoll.shape)

(128, 3000)


In [38]:
flipped = np.flip(pianoRoll, 0)
print(pianoRoll[:,0])
print(flipped[:,0])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]


In [32]:
for i in range(200):
        print(pianoRoll[:,i])
        #print(flipped[:,i])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [74]:
print(pianoRoll[:,3000])

[ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.         72.93514013
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0. 

In [56]:
v_changes = np.nonzero(np.diff(pianoRoll).T)

In [69]:
print(np.array(v_changes).shape)
print(np.array(v_changes)[0][6])

(2, 4179)
1


In [37]:
for i in range(200):
    print(pianoRolls[2][i])

IndexError: index 2 is out of bounds for axis 0 with size 1